# Lighthouse + Flow Deck Quick Start

This notebook demonstrates how to use the Crazyflie 2.1 brushless drone with both the Lighthouse Positioning System and Flow Deck for precise positioning and movement control.

**Features:**
- Continuous XYZ state estimates from Lighthouse positioning
- MotionCommander integration for smooth movement
- Hover functionality using Lighthouse position feedback
- Forward movement with real-time position monitoring

**Requirements:**
- Crazyflie 2.1 brushless drone
- Lighthouse Deck
- Flow Deck
- Lighthouse base stations (pre-configured)

Reference Documentation: 
* [Bitcraze User Guide - Logging](www.bitcraze.io/documentation/repository/crazyflie-clients-python/master/userguides/userguide_client/#logging)
* [Lighthouse Position System Home Page (bitcraze)](https://www.bitcraze.io/documentation/system/positioning/ligthouse-positioning-system/)
* [Extended Kalman Filters (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/functional-areas/sensor-to-control/state_estimators/#extended-kalman-filter)
* [State Estimation Logging (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/api/logs/#stateestimate)
* [Logging (bitcraze)](https://www.bitcraze.io/documentation/repository/crazyflie-clients-python/master/userguides/userguide_client/#logging)
* [Lighthouse Positioning System: Dataset, Accuracy, and Precision for UAV Research](https://whoenig.github.io/publications/2021_ICRA-Swarm-workshop_Taffanel.pdf)


## Imports


In [ ]:
# Standard libraries
import logging
import sys
import time
import threading
from threading import Event
from dotenv import load_dotenv
import os
import numpy as np

# Crazyflie
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper
from cflib.crazyflie.mem import LighthouseMemHelper

## Configuration and Global Variables


In [ ]:
# Load environment variables
load_dotenv()

# URI to the Crazyflie to connect to
URI = uri_helper.uri_from_env(default=os.getenv("RADIO_URI", 'radio://0/80/2M/E7E7E7E7E8'))

# Movement parameters
DEFAULT_HEIGHT = 1.0  # Default flight height in meters
HOVER_TOLERANCE = 0.05  # Position tolerance for hovering in meters
MOVEMENT_SPEED = 0.3  # Movement speed in m/s
UPDATE_RATE = 10  # Position update rate in Hz

# --- Position tracking global vars ---
current_position = {'x': 0.0, 'y': 0.0, 'z': 0.0}
target_position = {'x': 0.0, 'y': 0.0, 'z': DEFAULT_HEIGHT}
# Create a mutex (mutual-exclusion lock) to ensure thread-safe access to current_position
# (i.e. only one thread reads current_position at a time)
position_lock = threading.Lock()


# Events for deck detection
lighthouse_deck_event = Event()
flow_deck_event = Event()


# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


## Deck Detection Callbacks

In [ ]:
def param_deck_lighthouse(_, value_str):
    """
    Callback to check if the Lighthouse deck is attached.
    """
    value = int(value_str)
    if value:
        lighthouse_deck_event.set()
        print('Lighthouse deck is attached!')
    else:
        print('Lighthouse deck is NOT attached!')


def param_deck_flow(_, value_str):
    """
    Callback to check if the Flow deck is attached.
    """
    value = int(value_str)
    if value:
        flow_deck_event.set()
        print('Flow deck is attached!')
    else:
        print('Flow deck is NOT attached!')


## Position Logging and Control

In [ ]:
def log_position_callback(timestamp, data, logconf):
    """
    Callback function to receive and store position data from Lighthouse.
    """
    with position_lock:
        current_position['x'] = data['stateEstimate.x']
        current_position['y'] = data['stateEstimate.y']
        current_position['z'] = data['stateEstimate.z']
    
    # Print position for monitoring
    print(f"Position: X={current_position['x']:.3f}, Y={current_position['y']:.3f}, Z={current_position['z']:.3f}")

def get_current_position():
    """
    Get the current position in a thread-safe manner.
    """
    with position_lock:
        return current_position.copy()

def set_target_position(x, y, z):
    """
    Set a new target position.
    """
    global target_position
    target_position = {'x': x, 'y': y, 'z': z}
    print(f"Target set to: X={x:.3f}, Y={y:.3f}, Z={z:.3f}")


def is_at_target_position():
    """
    Check if the drone is within tolerance of the target position.
    """
    pos = get_current_position()
    dx = abs(pos['x'] - target_position['x'])
    dy = abs(pos['y'] - target_position['y'])
    dz = abs(pos['z'] - target_position['z'])
    
    return dx < HOVER_TOLERANCE and dy < HOVER_TOLERANCE and dz < HOVER_TOLERANCE


## Movement Functions


In [ ]:
def hover_at_position(scf, duration=5.0):
    """
    Hover at the current target position for a specified duration.
    """
    print(f"Hovering for {duration} seconds...")
    
    start_time = time.time()
    while time.time() - start_time < duration:
        if is_at_target_position():
            print("Position maintained within tolerance")
        else:
            pos = get_current_position()
            print(f"Position drift: X={pos['x']-target_position['x']:.3f}, Y={pos['y']-target_position['y']:.3f}, Z={pos['z']-target_position['z']:.3f}")
        
        time.sleep(0.1)


def move_forward_with_position_monitoring(mc, distance=0.5):
    """
    Move forward while continuously monitoring position. Uses an existing MotionCommander (mc).
    """
    print(f"Moving forward {distance}m while monitoring position...")
    
    # Get starting position
    start_pos = get_current_position()
    target_x = start_pos['x'] + distance
    
    # Set new target
    set_target_position(target_x, start_pos['y'], start_pos['z'])
    
    # Move forward using provided MotionCommander
    mc.forward(distance)
    
    # Monitor position during movement
    while not is_at_target_position():
        pos = get_current_position()
        remaining_distance = abs(target_x - pos['x'])
        print(f"Remaining distance: {remaining_distance:.3f}m")
        time.sleep(0.1)
    print("Forward movement completed!")


def takeoff_and_hover(mc, height=DEFAULT_HEIGHT):
    """
    Hover after MotionCommander takes off on context enter. Avoids double takeoff.
    """
    print(f"Taking off to {height}m and hovering...")
    
    # Get starting position
    start_pos = get_current_position()
    set_target_position(start_pos['x'], start_pos['y'], height)
    
    # MotionCommander context already took off to default_height
    # Optionally adjust height if different from default
    if abs(height - DEFAULT_HEIGHT) > 1e-3:
        if height > DEFAULT_HEIGHT:
            mc.up(height - DEFAULT_HEIGHT)
        else:
            mc.down(DEFAULT_HEIGHT - height)
    
    hover_at_position(None, 3.0)
    print("Takeoff and hover completed!")


def return_to_start(mc):
    """
    Return to the starting position using an existing MotionCommander (mc).
    """
    print("Returning to starting position...")
    
    # Set target to starting position (0, 0, DEFAULT_HEIGHT)
    set_target_position(0.0, 0.0, DEFAULT_HEIGHT)
    
    # Move to starting position
    pos = get_current_position()
    
    # Calculate required movements
    dx = -pos['x']
    dy = -pos['y']
    
    if abs(dx) > 0.1:
        if dx > 0:
            mc.forward(abs(dx))
        else:
            mc.back(abs(dx))
    
    if abs(dy) > 0.1:
        if dy > 0:
            mc.right(abs(dy))
        else:
            mc.left(abs(dy))
    
    print("Return to start completed!")


In [ ]:
def log_battery_once(scf):
    """
    Log battery status once (voltage and percentage) and print it.
    """
    from threading import Event
    evt = Event()
    battery = { 'vbat': None, 'percent': None }

    def _cb(timestamp, data, logconf):
        battery['vbat'] = data.get('pm.vbat')
        battery['percent'] = data.get('pm.batteryLevel')
        evt.set()

    try:
        logconf_bat = LogConfig(name='Battery', period_in_ms=200)
        logconf_bat.add_variable('pm.vbat', 'float')
        logconf_bat.add_variable('pm.batteryLevel', 'uint8_t')
        scf.cf.log.add_config(logconf_bat)
        logconf_bat.data_received_cb.add_callback(_cb)
        logconf_bat.start()
        evt.wait(timeout=2.0)
    except Exception as e:
        print(f"Battery log setup/read failed: {e}")
    finally:
        try:
            logconf_bat.stop()
        except Exception:
            pass

    if battery['vbat'] is not None or battery['percent'] is not None:
        if battery['vbat'] is not None and battery['percent'] is not None:
            print(f"Battery: {battery['vbat']:.2f} V  |  {battery['percent']} %")
        elif battery['vbat'] is not None:
            print(f"Battery: {battery['vbat']:.2f} V")
        else:
            print(f"Battery: {battery['percent']} %")
    else:
        print("Battery: unavailable (no data)")


## Main Flight Sequence


In [ ]:
def main_flight_sequence(scf):
    """
    Main flight sequence demonstrating Lighthouse + Flow Deck integration.
    Uses ONE MotionCommander context for the entire flight to avoid re-takeoff/land.
    """
    print("Starting main flight sequence...")
    
    try:
        with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
            # Fallback: if no lift after context enter, explicitly take off
            time.sleep(1.5)
            if get_current_position().get('z', 0.0) < 0.05:
                print("No lift detected after context enter, calling mc.take_off()...")
                mc.take_off(DEFAULT_HEIGHT)

            # 1. Take off and initial hover
            takeoff_and_hover(mc, DEFAULT_HEIGHT)
            
            # 2. Move forward with position monitoring
            move_forward_with_position_monitoring(mc, 0.5)
            
            # 3. Hover at new position
            hover_at_position(None, 5.0)
            
            # 4. Move forward again
            move_forward_with_position_monitoring(mc, 0.5)
            
            # 5. Hover again
            hover_at_position(None, 5.0)
            
            # 6. Return to starting position
            return_to_start(mc)
            
            # 7. Final hover
            hover_at_position(None, 2.0)
            
            # Exiting the context will land
        
        print("Flight sequence completed successfully!")
        
    except Exception as e:
        print(f"Error during flight sequence: {e}")
        # Emergency landing fallback
        try:
            with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
                mc.land()
        except Exception:
            pass


## Execute Flight


In [ ]:
# Initialize the low-level drivers
cflib.crtp.init_drivers()

print(f"Connecting to Crazyflie at: {URI}")

with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
    # Set up deck detection callbacks
    scf.cf.param.add_update_callback(group='deck', name='bcLighthouse4', cb=param_deck_lighthouse)
    scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)
    
    # Set up position logging
    logconf = LogConfig(name='Position', period_in_ms=int(1000/UPDATE_RATE))
    logconf.add_variable('stateEstimate.x', 'float')
    logconf.add_variable('stateEstimate.y', 'float')
    logconf.add_variable('stateEstimate.z', 'float')
    scf.cf.log.add_config(logconf)
    logconf.data_received_cb.add_callback(log_position_callback)
    
    # Wait for both decks to be detected
    print("Waiting for decks to be detected...")
    if not lighthouse_deck_event.wait(timeout=10):
        print('ERROR: Lighthouse deck not detected!')
        # Print out all deck params to help debug
        try:
            print("Deck params:")
            for p in scf.cf.param.toc.toc:
                if p.startswith('deck.'):
                    try:
                        print(p, scf.cf.param.get_value(p))
                    except Exception:
                        print(p, 'N/A')
        except Exception:
            pass
        sys.exit(1)
    
    if not flow_deck_event.wait(timeout=10):
        print('ERROR: Flow deck not detected!')
        sys.exit(1)
    
    print("Both decks detected! Starting position logging...\n")
    
    print("--- Battery Level ---")
    # Battery check before arming
    log_battery_once(scf)
    
    print("\n--- Position Logging ---")
    # Start position logging
    logconf.start()
    
    # Wait a moment for initial position data
    time.sleep(2)

    # Enable high-level commander and reset Kalman estimator
    try:
        scf.cf.param.set_value('commander.enHighLevel', '1')
    except Exception:
        pass
    try:
        scf.cf.param.set_value('stabilizer.estimator', '2')  # 2 = EKF
    except Exception:
        pass
    try:
        scf.cf.param.set_value('kalman.resetEstimation', '1')
        time.sleep(0.1)
        scf.cf.param.set_value('kalman.resetEstimation', '0')
    except Exception:
        pass
    time.sleep(0.5)
    
    # Arm the Crazyflie
    print("Arming Crazyflie...")
    scf.cf.platform.send_arming_request(True)
    time.sleep(1.0)
    
    # Execute main flight sequence
    main_flight_sequence(scf)
    
    # Stop logging
    logconf.stop()
    
    print("Flight completed!")


## Manual Control Functions

You can also use these functions individually for manual control:


In [ ]:
# Example: Manual takeoff and hover
# Uncomment the lines below to run individual functions

# with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
#     scf.cf.param.add_update_callback(group='deck', name='bcLighthouse4', cb=param_deck_lighthouse)
#     scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)
#     
#     logconf = LogConfig(name='Position', period_in_ms=20)
#     logconf.add_variable('stateEstimate.x', 'float')
#     logconf.add_variable('stateEstimate.y', 'float')
#     logconf.add_variable('stateEstimate.z', 'float')
#     scf.cf.log.add_config(logconf)
#     logconf.data_received_cb.add_callback(log_position_callback)
#     
#     if lighthouse_deck_event.wait(timeout=5) and flow_deck_event.wait(timeout=5):
#         logconf.start()
#         time.sleep(2)
#         scf.cf.platform.send_arming_request(True)
#         time.sleep(1.0)
#         
#         # Manual control examples:
#         # takeoff_and_hover(scf, 1.0)
#         # move_forward_with_position_monitoring(scf, 0.5)
#         # hover_at_position(scf, 5.0)
#         
#         logconf.stop()
#     else:
#         print('Required decks not detected!')


## Troubleshooting

**Common Issues:**

1. **Lighthouse deck not detected**: Ensure the Lighthouse deck is properly attached and the base stations are configured
2. **Flow deck not detected**: Check that the Flow deck is securely connected
3. **Position estimates not updating**: Verify that Lighthouse base stations are active and the drone is within range
4. **Connection issues**: Check your radio URI and ensure the Crazyflie is powered on

**Position Monitoring:**
- The notebook continuously logs XYZ position estimates from the Lighthouse system
- Position tolerance can be adjusted via the `HOVER_TOLERANCE` variable
- Update rate can be modified via the `UPDATE_RATE` variable

**Safety Features:**
- Emergency landing is triggered if errors occur during flight
- Position monitoring ensures movements are within expected ranges
- Deck detection prevents flight without required hardware
